In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import utils
from ewm import ewm_G

In [3]:
# Set the root path of the particle generator ewm experiments folder
exp_root = "/media/hdd1/kai/particle_generator/experiments/ewm_models/"

In [4]:
# Set the path where the G model generated code_vector data are to be stored
deploy_root = "/media/hdd1/kai/particle_generator/ewm_code_vectors/"

## Get the names of all the experiments in the exp_root folder

In [5]:
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

print("-"*60)
for i in range(len(exp_paths)):
    print("\n Exp_{}:".format(str(i)), exp_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model 

------------------------------------------------------------


## Select the experiment you want

In [6]:
exp_dir = exp_paths[0]

In [7]:
# Create the full path to the experiment
exp_path = os.path.join(exp_root, exp_dir) + "/"
print("Experiment path set as: \n{}".format(exp_path))

Experiment path set as: 
/media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/


In [8]:
# Path to model weights
weights_dir = "weights/"

In [9]:
# Set the GPU (GPU 1 is the best option)
device = torch.device(1)

## Load the config .csv as a dict
- Get the model architecture from the config_df

In [10]:
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [11]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].item())
n_hidden = int(config_df[config_df['Unnamed: 0'].str.contains("n_hidden")==True]['0'].item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].item())
z_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("z_dim")==True]['0'].item())
print("{} Layer model with {} hidden units per layer".format(n_layers, n_hidden))
print("Mapping {}_dim_Gaussian to {}_dim code vectors".format(z_dim, l_dim))

4 Layer model with 512 hidden units per layer
Mapping 100_dim_Gaussian to 20_dim code vectors


## Set up the Generator model

In [12]:
# Model kwargs
fc_sizes = [n_hidden] * n_layers
ewm_kwargs = { 'z_dim':z_dim, 'fc_sizes':fc_sizes, 'n_out':l_dim }

In [13]:
# Create generator on GPU
G = ewm_G(**ewm_kwargs).to(device)
G.weights_init()

## Load model checkpoint

In [14]:
# Get checkpoint name(s)
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [15]:
print("-"*60)
for i in range(len(checkpoint_names)):
    print("\n {} :".format(str(i)), checkpoint_names[i], '\n')
    print("-"*60)

------------------------------------------------------------

 0 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_99.tar 

------------------------------------------------------------

 1 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_0.tar 

------------------------------------------------------------

 2 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_1.tar 

------------------------------------------------------------


In [16]:
# Select the checkpoint you want
best_checkpoint = checkpoint_names[0]

In [17]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(best_checkpoint)

In [18]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
G.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [19]:
# Put the model in evaluation mode
G.eval()

ewm_G(
  (fc): Sequential(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
  )
  (out): Sequential(
    (0): Linear(in_features=512, out_features=20, bias=True)
  )
)

## Set the directory where the model outputs should be saved

In [20]:
deploy_dir = deploy_root + "ewm_code_vectors_{}_{}/".format(im_size, l_dim)

In [ ]:
os.mkdir(deploy_dir) # Make first level folder

In [21]:
deploy_dir = deploy_dir + "ewm_code_vectors_{}_{}/".format(im_size, l_dim)

In [ ]:
os.mkdir(deploy_dir) # Make second level folder

In [22]:
print("Deploy samples will be saved to {}".format(deploy_dir))

Deploy samples will be saved to /media/hdd1/kai/particle_generator/ewm_code_vectors/ewm_code_vectors_64_20/ewm_code_vectors_64_20/


## Create a set of code_vectors

In [25]:
code_vectors = []; num_vectors = 100
for idx in range(num_vectors):
    z_rand = torch.randn(1, z_dim).to(device)
    code_vector = G(z_rand)
    code_vectors.append(code_vector.detach().cpu().numpy())
print("Generated {} code vectors".format(len(code_vectors)))

Generated 100 code vectors


In [26]:
type(code_vectors[0])

numpy.ndarray

## Save the list of ewm generated code_vectors to disk

In [27]:
for idx, arr in enumerate(code_vectors):
    arr_name = deploy_dir + "target_{}.npy".format(idx)
    np.save(arr_name, arr)

In [28]:
# Load a test vector
x = np.load(deploy_dir + "target_0.npy")
print(type(x))
print(x)

<class 'numpy.ndarray'>
[[-30.92391     -3.0897214  -22.233782    20.81274    -13.930739
  -12.187764   -12.697512   -17.915503    -8.4963455   23.620121
   11.477194   -34.05378    -20.36834     -5.9694304   -0.17459583
   -5.945078    -9.21459      2.2330112   31.371498   -13.175336  ]]
